In [10]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl, re
%matplotlib inline

# Preprocessing Energy Dataset

In [11]:
#Read energy dataset
edf = pd.read_excel('./datasets/SES_Public_2021_tidy.xlsx', sheet_name='T3.5')

In [12]:
#Rename cols names
edf = edf.rename(columns={'Description': 'Towns', 'year': 'Year', 'month': 'Month', 'dwelling_type': 'Dwelling Type', 'kwh_per_acc':'Avg kWh'})
edf

,Dwelling Type,Year,Month,Region,Towns,Avg kWh
0,1-room / 2-room,2005.0,1,Central Region,Bishan,104.9
1,1-room / 2-room,2005.0,1,Central Region,Bukit Merah,140.7
2,1-room / 2-room,2005.0,1,Central Region,Central Region,136.5
3,1-room / 2-room,2005.0,1,Central Region,Geylang,148.5
4,1-room / 2-room,2005.0,1,Central Region,Kallang,115.6
...,...,...,...,...,...,...
78666,Public Housing,2021.0,Annual,West Region,Jurong West,386.7
78667,Public Housing,2021.0,Annual,West Region,West Region,367
78668,NaN,NaN,NaN,NaN,NaN,NaN
78669,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#Remove last last 3 rows as empty and irrelevant
edf = edf.iloc[:-3]

In [14]:
#Remove irrelevant/aggregated rows
edf = edf[edf['Month']!='Annual']
edf = edf[edf['Region']!='Overall']
edf = edf[edf['Dwelling Type']!='Overall']
edf = edf[edf['Towns']!='Overall']
edf = edf[edf['Avg kWh']!='s']

# Preprocessing Weather Dataset

In [15]:
#Read weather dataset
wdf = pd.read_csv("./datasets/weather_data.csv")

In [16]:
#Remove day as data is be too granular for our use case, will avg based on region, year, month instead
del wdf['Day']
#Rename cols to match elec data as will be joining base on: Towns, Year, Month
wdf = wdf.rename(columns={'Station': 'Towns'})

In [17]:
#Exploring the weather data, realised there are same town names just (east) and (west) versions
#Thus, aim to find such towns and combine together and take the avg metrics
towns = wdf.Towns.unique()
towns_with_brac = []
for town in towns:
    if "(" in town:
        towns_with_brac.append(town)
towns_with_brac.sort()

print("Original Towns")
print(towns_with_brac,"\n")

#Remove the brackets and combine into unique list of dup towns
unique_towns_with_brac = []
for town in towns_with_brac:
    unique_towns_with_brac.append(re.sub("([\(\[]).*?([\)\]])", "", town).strip())
unique_towns_with_brac = list(dict.fromkeys(unique_towns_with_brac))

print("Preprocessed Towns")
print(unique_towns_with_brac)

#Replace towns 
for town in unique_towns_with_brac:
    wdf.loc[wdf['Towns'].str.contains(town, case=False), 'Towns'] = town

Original Towns
['Boon Lay (East)', 'Boon Lay (West)', 'Choa Chu Kang (Central)', 'Choa Chu Kang (South)', 'Choa Chu Kang (West)', 'Jurong (East)', 'Jurong (North)', 'Jurong (West)', 'Pasir Ris (Central)', 'Pasir Ris (West)', 'Somerset (Road)'] 

Preprocessed Towns
['Boon Lay', 'Choa Chu Kang', 'Jurong', 'Pasir Ris', 'Somerset']


In [18]:
#Use the groupby function to avg out the dup towns metrics data
wdf = wdf.groupby(['Towns', 'Year', 'Month']).mean()
wdf

Daily Rainfall Total (mm)  \
Towns     Year   Month                              
Admiralty 2009.0 1.0                     0.080000   
                 2.0                     5.285714   
                 3.0                    11.600000   
                 4.0                     4.960000   
                 5.0                     6.632258   
...                                           ...   
Yishun    2020.0 12.0                    8.025806   
          2021.0 1.0                    16.268966   
                 2.0                     1.207143   
                 3.0                     7.466667   
                 4.0                          NaN   

                        Highest 30 min Rainfall (mm)  \
Towns     Year   Month                                 
Admiralty 2009.0 1.0                             NaN   
                 2.0                             NaN   
                 3.0                             NaN   
                 4.0                             NaN   
                 5.0                             NaN   
...                                              ...   
Yishun    2020.0 12.0                       5.967742   
          2021.0 1.0                        4.075862   
                 2.0                        0.935714   
                 3.0                        4.560000   
                 4.0                             NaN   

                        Highest 60 min Rainfall (mm)  \
Towns     Year   Month                                 
Admiralty 2009.0 1.0                             NaN   
                 2.0                             NaN   
                 3.0                             NaN   
                 4.0                             NaN   
                 5.0                             NaN   
...                                              ...   
Yishun    2020.0 12.0                       6.483871   
          2021.0 1.0                        5.262069   
                 2.0                        0.957143   
                 3.0                        5.520000   
                 4.0                             NaN   

                        Highest 120 min Rainfall (mm)  Mean Temperature (°C)  \
Towns     Year   Month                                                         
Admiralty 2009.0 1.0                              NaN              26.342857   
                 2.0                              NaN              26.767857   
                 3.0                              NaN              26.913793   
                 4.0                              NaN              28.120000   
                 5.0                              NaN              28.483871   
...                                               ...                    ...   
Yishun    2020.0 12.0                        7.174194                    NaN   
          2021.0 1.0                         7.289655                    NaN   
                 2.0                         1.064286                    NaN   
                 3.0                         6.466667                    NaN   
                 4.0                              NaN                    NaN   

                        Maximum Temperature (°C)  Minimum Temperature (°C)  \
Towns     Year   Month                                                       
Admiralty 2009.0 1.0                   30.510000                 23.990000   
                 2.0                   31.442857                 24.260714   
                 3.0                   32.093333                 24.020000   
                 4.0                   32.196667                 25.066667   
                 5.0                   32.590323                 25.093548   
...                                          ...                       ...   
Yishun    2020.0 12.0                        NaN                       NaN   
          2021.0 1.0                         NaN                       NaN   
                 2.0                         NaN       

# Merging Of Datasets

In [19]:
#Merge the 2 dataframes base on Towns, Year and Month with elec data as left df
newdf = edf.merge(wdf, on=["Towns","Year","Month"], how="left")
#Rename for easy function calls in the lower section
df = newdf.copy()
df

,Dwelling Type,Year,Month,Region,Towns,Avg kWh,Daily Rainfall Total (mm),Highest 30 min Rainfall (mm),Highest 60 min Rainfall (mm),Highest 120 min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h)
0,1-room / 2-room,2005.0,1,Central Region,Bishan,104.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1-room / 2-room,2005.0,1,Central Region,Bukit Merah,140.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1-room / 2-room,2005.0,1,Central Region,Central Region,136.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1-room / 2-room,2005.0,1,Central Region,Geylang,148.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1-room / 2-room,2005.0,1,Central Region,Kallang,115.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60563,Public Housing,2021.0,6,West Region,Choa Chu Kang,470.8,9.800000,6.088636,7.811364,9.379545,28.10,32.660000,24.540000,8.626667,31.866667
60564,Public Housing,2021.0,6,West Region,Clementi,360.1,7.493333,4.420000,5.413333,6.740000,28.24,32.076667,25.193333,4.853333,26.540000
60565,Public Housing,2021.0,6,West Region,Jurong East,434.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60566,Public Housing,2021.0,6,West Region,Jurong West,435.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
#Since the weather data has missing data for the electricity data, fill the missing data with the avg of 
#the groupby by 3 iterations: 1. Region, Year, Month | 2. Region, Year | 3. Region
#The reason is that the weather data based of regions is applicable in different towns due to their proximity
#The year and month data are good to have to identify the time.

#Get list of col names 
df_col_names = list(df.columns)
metrics_val_cols = df_col_names[6:]

#fill the nan values of the metrics cols (which are weather data) with the avg as stated above
temp_list = []
for col in metrics_val_cols:
    df[col] = df.groupby(['Region','Year','Month'])[col].apply(lambda x:x.fillna(x.mean()))
    r1 = df[col].isna().sum()
    df[col] = df.groupby(['Region','Year'])[col].apply(lambda x:x.fillna(x.mean()))
    r2 =df[col].isna().sum()
    df[col] = df.groupby('Region')[col].apply(lambda x:x.fillna(x.mean()))
    r3 = df[col].isna().sum()
    temp = f"{col} NaNs| iter1={r1} | iter2={r2} | iter3={r3}"
    print(temp)
df

Daily Rainfall Total (mm) NaNs| iter1=0 | iter2=0 | iter3=0
Highest 30 min Rainfall (mm) NaNs| iter1=32931 | iter2=32931 | iter3=0
Highest 60 min Rainfall (mm) NaNs| iter1=32931 | iter2=32931 | iter3=0
Highest 120 min Rainfall (mm) NaNs| iter1=32931 | iter2=32931 | iter3=0
Mean Temperature (°C) NaNs| iter1=12993 | iter2=10960 | iter3=0
Maximum Temperature (°C) NaNs| iter1=11250 | iter2=9402 | iter3=0
Minimum Temperature (°C) NaNs| iter1=11250 | iter2=9402 | iter3=0
Mean Wind Speed (km/h) NaNs| iter1=11805 | iter2=9624 | iter3=0
Max Wind Speed (km/h) NaNs| iter1=16409 | iter2=13803 | iter3=0


,Dwelling Type,Year,Month,Region,Towns,Avg kWh,Daily Rainfall Total (mm),Highest 30 min Rainfall (mm),Highest 60 min Rainfall (mm),Highest 120 min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h)
0,1-room / 2-room,2005.0,1,Central Region,Bishan,104.9,2.922581,3.583618,4.428546,5.007326,27.482216,31.279255,25.073645,7.177215,35.345983
1,1-room / 2-room,2005.0,1,Central Region,Bukit Merah,140.7,2.922581,3.583618,4.428546,5.007326,27.482216,31.279255,25.073645,7.177215,35.345983
2,1-room / 2-room,2005.0,1,Central Region,Central Region,136.5,2.922581,3.583618,4.428546,5.007326,27.482216,31.279255,25.073645,7.177215,35.345983
3,1-room / 2-room,2005.0,1,Central Region,Geylang,148.5,2.922581,3.583618,4.428546,5.007326,27.482216,31.279255,25.073645,7.177215,35.345983
4,1-room / 2-room,2005.0,1,Central Region,Kallang,115.6,2.922581,3.583618,4.428546,5.007326,27.482216,31.279255,25.073645,7.177215,35.345983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60563,Public Housing,2021.0,6,West Region,Choa Chu Kang,470.8,9.800000,6.088636,7.811364,9.379545,28.100000,32.660000,24.540000,8.626667,31.866667
60564,Public Housing,2021.0,6,West Region,Clementi,360.1,7.493333,4.420000,5.413333,6.740000,28.240000,32.076667,25.193333,4.853333,26.540000
60565,Public Housing,2021.0,6,West Region,Jurong East,434.2,8.486667,5.280657,6.481566,7.877626,28.170000,32.368333,24.866667,6.740000,29.203333
60566,Public Housing,2021.0,6,West Region,Jurong West,435.8,8.486667,5.280657,6.481566,7.877626,28.170000,32.368333,24.866667,6.740000,29.203333


In [21]:
#Checking dataset has no null values
df.isna().sum()

Dwelling Type                    0
Year                             0
Month                            0
Region                           0
Towns                            0
Avg kWh                          0
Daily Rainfall Total (mm)        0
Highest 30 min Rainfall (mm)     0
Highest 60 min Rainfall (mm)     0
Highest 120 min Rainfall (mm)    0
Mean Temperature (°C)            0
Maximum Temperature (°C)         0
Minimum Temperature (°C)         0
Mean Wind Speed (km/h)           0
Max Wind Speed (km/h)            0
dtype: int64

In [22]:
#round vals as 1 dp
val_cols = df_col_names[6:]
for col in val_cols:
    df[col] = df[col].round(1)

In [24]:
#If df no longer contain nan values, save the combined dataset
if (df.isnull().values.any()) == True:
    print("Missing value, please fix it first")
else:
    df.to_excel("./datasets/combined.xlsx")
    print("Successfully cleaned dataset and saved it as 'combined.xlsx'")

Successfully cleaned dataset and saved it as 'combined.xlsx'


# EDA of merged dataset

In [ ]:
temp_df = df[["Avg kWh", "Region", "Year", "Month", "Towns", "Dwelling Type"]]
pp = sns.pairplot(temp_df, diag_kind="hist")
pp.savefig("pp1.png") 
pp

In [ ]:
pp = sns.pairplot(df[["Avg kWh", "Daily Rainfall Total (mm)", "Highest 30 min Rainfall (mm)", "Highest 60 min Rainfall (mm)", "Highest 120 min Rainfall (mm)"]], diag_kind="hist")
pp.savefig("pp2.png") 
pp

In [ ]:
pp = sns.pairplot(df[["Avg kWh", "Mean Temperature (°C)", "Maximum Temperature (°C)", "Minimum Temperature (°C)", "Mean Wind Speed (km/h)", "Max Wind Speed (km/h)"]], diag_kind="hist")
pp.savefig("pp3.png") 
pp